In [26]:
class opt():
    hparamDatasetPath = r"/Users/nikolai/Downloads/UPC/VSC/Project/dataset/sequences",
    hparamYamlConfigPath = "/Users/nikolai/Downloads/UPC/VSC/Project/Workspace-AD-LiDAR/F0_Visualization/semantic-kitti-api/config/semantic-kitti.yaml",
    hparamNumberOfRandomPoints = 4000
    hparamDatasetSequence = '04'    
    hparamNumberOfClasses = 4
    hparamBatchSize = 32
    hparamNumberOfEpochs = 250 #TODO: add to config ?
    hparamNumberOfWorkers = 4 #TODO: add to config ?
    hparamOutputFolder = 'cls' #TODO: add to config ?
    hparamDeviceType = 'cuda'

In [31]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from B0_Dataset.dataset import SemanticKittiDataset
from D0_Modeling.model import PointNetCls, feature_transform_regularizer
from A0_Configuration.hyperparam import Parsing
import torch.nn.functional as F
from tqdm import tqdm
from B1_Dataloader.dataloader import DataLoader_

blue = lambda x: '\033[94m' + x + '\033[0m'

#opt.manualSeed = random.randint(1, 10000)
#print("Random Seed: ", opt.manualSeed)
#random.seed(opt.manualSeed)
torch.manual_seed(123)

train_dataset = SemanticKittiDataset(
    data_catalog_path=opt.hparamDatasetPath[0],
    sequence_number=opt.hparamDatasetSequence,
    yaml_config_path=opt.hparamYamlConfigPath[0],
    n_points=opt.hparamNumberOfRandomPoints,
    action_type='train')

test_dataset = SemanticKittiDataset(
    data_catalog_path=opt.hparamDatasetPath[0],
    sequence_number=opt.hparamDatasetSequence,
    yaml_config_path=opt.hparamYamlConfigPath[0],
    n_points=opt.hparamNumberOfRandomPoints,
    action_type='test')

train_dataloader = DataLoader_(
    dataset= train_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True,
    num_workers=int(opt.hparamNumberOfWorkers)
)
test_dataloader = DataLoader_(
    dataset = test_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True,
    num_workers=int(opt.hparamNumberOfWorkers))

print(len(train_dataset), len(test_dataset))

try:
    os.makedirs(opt.hparamOutputFolder)
except OSError:
    pass

271 271


In [30]:

# classifier = PointNetCls(k=num_classes, feature_transform=opt.feature_transform)
classifier = PointNetCls(k=opt.hparamNumberOfClasses, feature_transform=False)

# if opt.model != '':
#     classifier.load_state_dict(torch.load(opt.model)) #TODO: use if we plan to load model


optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
classifier = classifier.to(opt.hparamDeviceType)

num_batch = len(train_dataset) / opt.hparamBatchSize #FIXME: is it ok?


for epoch in range(opt.hparamNumberOfEpochs):
    scheduler.step()
    for i, data in enumerate(train_dataloader):
        points, target = data
        target = target[:, 0]
        points = points.transpose(2, 1)
        # points, target = points.cuda(), target.cuda()
        points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        loss = F.nll_loss(pred, target)
        # loss = nn.NNLLoss(pred, target)
        # if opt.feature_transform:
        #     loss += feature_transform_regularizer(trans_feat) * 0.001 :FIXME If we like to use feature transform
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cuda().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item() / float(opt.hparamBatchSize)))

        if i % 10 == 0:
            j, data = next(enumerate(test_dataloader, 0))
            points, target = data
            target = target[:, 0]
            points = points.transpose(2, 1)
            # points, target = points.cuda(), target.cuda()
            points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
            classifier = classifier.eval()
            pred, _, _ = classifier(points)
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cuda().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.hparamBatchSize)))

    torch.save(classifier.state_dict(), '%s/cls_model_%d.pth' % (opt.hparamOutputFolder, epoch))

total_correct = 0
total_testset = 0
for i,data in tqdm(enumerate(test_dataloader, 0)):
    points, target = data
    target = target[:, 0]
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    total_correct += correct.item()
    total_testset += points.size()[0]

print("final accuracy {}".format(total_correct / float(total_testset)))

AssertionError: Torch not compiled with CUDA enabled

In [7]:
train_dataloader.__class__

B1_Dataloader.dataloader.DataLoader_